In [313]:
import datetime
print(datetime.datetime.now())

2020-07-19 09:29:54.902973


In [381]:
import pandas as pd
import numpy as np
import iso8601 # need this for handling ISO8601 timestamps

In [383]:
#cnoc_test = pd.read_csv('/c/Users/rick_/Google Drive/E.Personal/MCUR/Projects/P04.cnoc/cnoc-test2.csv')
cnoc_test = pd.read_csv('/home/rbarthel/git/cnoc/results.csv',parse_dates=True)

Create the Index from the timestamp and sort for later use. Strictly, this is not necessary if not using rolling windows. The incoming timestamp , with an object data type, is an ISO8601 timestamp and this is converted to a nornmal DateTime before being made into an index.

In [384]:
cnoc_test['timestamp']=cnoc_test['timestamp'].apply(lambda x: iso8601.parse_date(x))

In [385]:
cnoc_test.set_index('timestamp', inplace=True)
cnoc_test = cnoc_test.sort_index()

In [386]:
cnoc_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 233463 entries, 2020-03-12 23:59:59.760000+00:00 to 2020-03-13 00:51:33.588000+00:00
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   mac         233463 non-null  object 
 1   latitude    233463 non-null  float64
 2   longtitude  233463 non-null  float64
dtypes: float64(2), object(1)
memory usage: 7.1+ MB


In [387]:
cnoc_test['latitude'].round(3)

timestamp
2020-03-12 23:59:59.760000+00:00    52.13
2020-03-12 23:59:59.760000+00:00    52.13
2020-03-12 23:59:59.761000+00:00    52.13
2020-03-12 23:59:59.761000+00:00    52.13
2020-03-12 23:59:59.765000+00:00    52.13
                                    ...  
2020-03-13 00:51:33.481000+00:00    52.13
2020-03-13 00:51:33.489000+00:00    52.13
2020-03-13 00:51:33.493000+00:00    52.13
2020-03-13 00:51:33.498000+00:00    52.13
2020-03-13 00:51:33.588000+00:00    52.13
Name: latitude, Length: 233463, dtype: float64

## Rolling Time Windows
Rolling time windows are needed just in case events occur at the edges of each window. For example, event 1 2020-07-18 12:04:30 and event 2 2020-07-18 12:05:10.

In a ````pandas```` dataframe it should be possible to do this with a rolling data frame

## Location Proximity
Location is similar to time. If lat/lon is rounded to 3 decimal places locations may be in close proximity but would appear on different polygons. To overcome this it must be possible to develop an 'edge' through proximity.

## Developing the Edges
In order to deduce the edges we would need to:
    1. Establish the list of nodes in the time window
    2. For each node calculate the proximity between every other observed node
    3. Discard any nodes outside the threshold of proximity
    4. Create an edge based on the proximity between the two nodes and use the distance as the weighting (possibly think about the impact of time as an additional weighting factor and add that as an additional weighting attribute)

## Calculate distance between 2 points
Probably stick this into a lambda function when we start with the pandas dataframe

In [388]:
from geopy.distance import great_circle

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(great_circle(coords_1, coords_2).km)

278.45856843965976


In [389]:
cnoc_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 233463 entries, 2020-03-12 23:59:59.760000+00:00 to 2020-03-13 00:51:33.588000+00:00
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   mac         233463 non-null  object 
 1   latitude    233463 non-null  float64
 2   longtitude  233463 non-null  float64
dtypes: float64(2), object(1)
memory usage: 7.1+ MB


In [390]:
import networkx as nx
MACnw = nx.Graph()

In [391]:
print(cnoc_test.index.dtype)

datetime64[ns, UTC]


## This block took ages to process and was interrupted.
Problem here is the use of a manual sliding window over the whole data set and then looking for each relationship in the whole dataset to create the edges.


In [394]:
for index, row in cnoc_test.iterrows():
## print(row['mac-address'], row['location'], index, index - pd.Timedelta('2m30s'), index + pd.Timedelta('2m30s'))
    curr_MAC = row['mac']
    MACnw.add_node(curr_MAC) #, MAC_label=row['mac-label'], locn=row['location'])
    curr_lat = row['latitude']
    curr_lon = row['longtitude']
    win_early = index - pd.Timedelta('2m30s')
    win_late = index + pd.Timedelta('2m30s')
    curr_set = cnoc_test[(cnoc_test['mac'] != curr_MAC) &\
                         (cnoc_test.index >= win_early) &\
                         (cnoc_test.index <= win_late)]
    ##curr_set = curr_set.apply(lambda x: great_circle((curr_lat, curr_lon), (x['latitude'],x['longitude'])).km, axis=1)
    #curr_set = curr_set[]
    for i1, r1 in curr_set.iterrows():
        prox = great_circle((curr_lat, curr_lon), (r1['latitude'],r1['longtitude'])).m
        window=i1.strftime('%X%x')
        if prox < 100 :
            MACnw.add_node(r1['mac']) #, MAC_label=r1['mac-label'], locn=r1['location'])
                #node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
                #node["value"] = len(neighbor_map[node["id"]])
            MACnw.add_edge(curr_MAC, r1['mac'], weight=prox, twindow=window)

KeyboardInterrupt: 

In [395]:
from pyvis.network import Network
net = Network(notebook=True)

In [396]:
net.from_nx(MACnw)
net.show("MACs.html")

In [399]:
print(MACnw.number_of_nodes())

99


In [400]:
print(MACnw.number_of_edges())

4074


In [401]:
print(nx.info(MACnw))

Name: 
Type: Graph
Number of nodes: 99
Number of edges: 4074
Average degree:  82.3030


In [ ]:
for edges in MACnw.edges :
    print(edges)

In [402]:
graphdf = nx.to_pandas_dataframe(MACnw)

AttributeError: module 'networkx' has no attribute 'to_pandas_dataframe'

## [Clustering Coefficient](https://www.datacamp.com/community/tutorials/social-network-analysis-python)

It is observed that people who share connections in a social network tend to form associations. In other words, there is a tendency in a social network to form clusters. We can determine the clusters of a node, Local Clustering Coefficient, which is the fraction of pairs of the node's friends (that is connections) that are connected with each other. To determine the local clustering coefficient, we make use of nx.clustering(Graph, Node) function.

In [ ]:
import networkx as nx
simpleMACnw = nx.Graph()

for index, row in cnoc_test.iterrows():
## print(row['mac-address'], row['location'], index, index - pd.Timedelta('2m30s'), index + pd.Timedelta('2m30s'))
    curr_MAC = row['mac']
    MACnw.add_node(curr_MAC) #, MAC_label=row['mac-label'], locn=row['location'])
    curr_lat = row['latitude']
    curr_lon = row['longtitude']
    win_early = index - pd.Timedelta('2m30s')
    win_late = index + pd.Timedelta('2m30s')
    curr_set = cnoc_test[(cnoc_test['mac'] != curr_MAC) &\
                         (cnoc_test.index >= win_early) &\
                         (cnoc_test.index <= win_late)]
    ##curr_set = curr_set.apply(lambda x: great_circle((curr_lat, curr_lon), (x['latitude'],x['longitude'])).km, axis=1)
    #curr_set = curr_set[]
    for i1, r1 in curr_set.iterrows():
        prox = great_circle((curr_lat, curr_lon), (r1['latitude'],r1['longtitude'])).m
        window=i1.strftime('%X%x')
        if prox < 100 :
            MACnw.add_node(r1['mac']) #, MAC_label=r1['mac-label'], locn=r1['location'])
                #node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
                #node["value"] = len(neighbor_map[node["id"]])
            MACnw.add_edge(curr_MAC, r1['mac'], weight=prox, twindow=window)

In [403]:
print(nx.edges(MACnw))

[('48:6D:BB:87:56:2B', '86:25:19:29:18:A7'), ('48:6D:BB:87:56:2B', '44:E9:DD:5E:6E:02'), ('48:6D:BB:87:56:2B', '44:E9:DD:F1:C7:19'), ('48:6D:BB:87:56:2B', 'F0:86:20:3F:05:75'), ('48:6D:BB:87:56:2B', 'CA:3A:6B:34:31:2F'), ('48:6D:BB:87:56:2B', '92:72:82:87:C8:7F'), ('48:6D:BB:87:56:2B', '44:E9:DD:F1:C7:17'), ('48:6D:BB:87:56:2B', '44:E9:DD:5E:6E:07'), ('48:6D:BB:87:56:2B', '44:E9:DD:5E:6E:05'), ('48:6D:BB:87:56:2B', 'B2:72:82:87:C8:7F'), ('48:6D:BB:87:56:2B', '90:72:82:87:C8:7F'), ('48:6D:BB:87:56:2B', 'B8:EE:0E:4E:FB:8E'), ('48:6D:BB:87:56:2B', '90:4D:4A:50:AE:61'), ('48:6D:BB:87:56:2B', '64:CC:22:9A:65:D0'), ('48:6D:BB:87:56:2B', '62:CC:22:9A:65:D4'), ('48:6D:BB:87:56:2B', 'EC:F4:51:A6:25:40'), ('48:6D:BB:87:56:2B', '30:FD:38:AC:F7:37'), ('48:6D:BB:87:56:2B', 'AC:3B:77:14:D4:40'), ('48:6D:BB:87:56:2B', 'AE:3B:77:14:D6:41'), ('48:6D:BB:87:56:2B', 'C0:3E:0F:11:0F:4F'), ('48:6D:BB:87:56:2B', '20:47:ED:A8:22:B2'), ('48:6D:BB:87:56:2B', '98:D8:63:A7:C5:04'), ('48:6D:BB:87:56:2B', 'B0:19:C6

In [404]:
G = nx.Graph()
nx.add_path(G, [1, 2, 3], color='red')
color = nx.get_edge_attributes(G, 'color')


In [405]:
print(nx.edges(G))

[(1, 2), (2, 3)]


In [406]:
print(color)

{(1, 2): 'red', (2, 3): 'red'}


In [417]:
cnoc_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233463 entries, 0 to 233462
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   mac         233463 non-null  object 
 1   latitude    233463 non-null  float64
 2   longtitude  233463 non-null  float64
dtypes: float64(2), object(1)
memory usage: 5.3+ MB


In [410]:
cnoc_nt = cnoc_test

In [413]:
cnoc_nt.reset_index(drop=True, inplace=True)

In [416]:
cnoc_nt.index


RangeIndex(start=0, stop=233463, step=1)

In [418]:
cnoc_nt['location']=list(zip(cnoc_nt['latitude'],cnoc_nt['longtitude']))

In [419]:
cnoc_nt.head()

,mac,latitude,longtitude,location
0,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)"
1,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)"
2,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)"
3,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)"
4,86:25:19:29:18:A7,52.130274,-2.749344,"(52.130274, -2.749344)"


In [420]:
cnoc_nt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233463 entries, 0 to 233462
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   mac         233463 non-null  object 
 1   latitude    233463 non-null  float64
 2   longtitude  233463 non-null  float64
 3   location    233463 non-null  object 
dtypes: float64(2), object(2)
memory usage: 7.1+ MB


In [421]:
cnoc_nt2 = cnoc_nt

In [422]:
cnoc_nt2.index[:]= cnoc_nt.index[:] = 0

TypeError: Index does not support mutable operations

In [425]:
cnoc_nt.index.dtype

dtype('int64')

In [429]:
cnoc_nt['key'] = cnoc_nt.index
cnoc_nt2['key'] = cnoc_nt2.index

In [430]:
cnoc_nt.head()

,mac,latitude,longtitude,location,key
0,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)",0
1,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)",1
2,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)",2
3,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)",3
4,86:25:19:29:18:A7,52.130274,-2.749344,"(52.130274, -2.749344)",4


In [432]:
cnoc_xc = pd.merge(cnoc_nt, cnoc_nt2, on = 'key')

In [435]:
cnoc_xc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233463 entries, 0 to 233462
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   mac_x         233463 non-null  object 
 1   latitude_x    233463 non-null  float64
 2   longtitude_x  233463 non-null  float64
 3   location_x    233463 non-null  object 
 4   key           233463 non-null  int64  
 5   mac_y         233463 non-null  object 
 6   latitude_y    233463 non-null  float64
 7   longtitude_y  233463 non-null  float64
 8   location_y    233463 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 17.8+ MB


In [436]:
cnoc_xc

,mac_x,latitude_x,longtitude_x,location_x,key,mac_y,latitude_y,longtitude_y,location_y
0,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)",0,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)"
1,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)",1,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)"
2,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)",2,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)"
3,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)",3,48:6D:BB:87:56:2B,52.130274,-2.749344,"(52.130274, -2.749344)"
4,86:25:19:29:18:A7,52.130274,-2.749344,"(52.130274, -2.749344)",4,86:25:19:29:18:A7,52.130274,-2.749344,"(52.130274, -2.749344)"
...,...,...,...,...,...,...,...,...,...
233458,E2:48:AF:01:0C:A7,52.130286,-2.749342,"(52.130286, -2.749342)",233458,E2:48:AF:01:0C:A7,52.130286,-2.749342,"(52.130286, -2.749342)"
233459,E0:48:AF:01:8C:DB,52.130286,-2.749342,"(52.130286, -2.749342)",233459,E0:48:AF:01:8C:DB,52.130286,-2.749342,"(52.130286, -2.749342)"
233460,E0:48:AF:01:8C:DB,52.130286,-2.749342,"(52.130286, -2.749342)",233460,E0:48:AF:01:8C:DB,52.130286,-2.749342,"(52.130286, -2.749342)"
233461,E2:48:AF:01:0C:A7,52.130286,-2.749342,"(52.130286, -2.749342)",233461,E2:48:AF:01:0C:A7,52.130286,-2.749342,"(52.130286, -2.749342)"
